# Word Segmentation and Association

![](../figs/intro_nlp/words/entelecheia_associaltion_vs_segmentation.png)

## Word Segmentation

- **Word segmentation** is the task of splitting a string of characters into words.
- Word segmentation is important for a machine to understand the meaning of a sentence.
- In English, we can split a string of characters into words by spaces.
- However, in languages like Chinese and Janpanese, there is no space between words.  
- Even in English, there are some cases where no space is used between words.
- Humans can easily segment a string of characters into words, even though there is no space between words.
- For example, we can easily segment the string of characters `Ilikechocolate` into words `I like chocolate`.

## Why should we segment words?

There are many applications that require word segmentation, even in English.

- Normalizing English compound nouns that are variably written for search engines.
  - For example, `ice cream` and `ice-cream` should be segmented into `icecream`.
- Word segmentation for compounds: Both orginal words and split words should be in the dictionary.
- Typing errors may be corrected by word segmentation.
- Conversion errors: During conversion, some spaces may be lost.
- OCR errors: OCRed text may contain errors.
- Keyword extraction from URL addresses, domain names, table column description or programming variables that are written without spaces.
- For password analysis, the extraction of terms from passwords can be required.
- Automatic CamelCasing of programming variables.
- Speech recognition: Speech recognition systems may not properly recognize spaces between words.


## Generating segment variants

We can generate all possible segment variants of a string of characters. Each distinct segment variant is called a **composition**.

- En a string of length $n$, there are $n-1$ possible positions to split the string.
- Each of the $n-1$ positions can be used as word boundary.
- Therefore, there are $2^{n-1}$ possible compositions.

The compositions have to be evaluated to find the best segmentation.

- The best segmentation is the one that has the highest probability.


### Naive Recursive Algorithm

- The naive recursive algorithm is to generate all possible compositions and evaluate them.
- The time complexity of the naive recursive algorithm is $O(2^n)$.
- The naive recursive algorithm is not efficient for long strings.

In [6]:
from pprint import pprint

def segment_naive(string):
    if not string:
        return []
    else:
        return [[string]] + [
            [string[:i]] + rest
            for i in range(1, len(string))
            for rest in segment_naive(string[i:])
        ]

In [7]:
pprint(segment_naive("isit"))

[['isit'],
 ['i', 'sit'],
 ['i', 's', 'it'],
 ['i', 's', 'i', 't'],
 ['i', 'si', 't'],
 ['is', 'it'],
 ['is', 'i', 't'],
 ['isi', 't']]


In [8]:
pprint(segment_naive("가방에"))

[['가방에'], ['가', '방에'], ['가', '방', '에'], ['가방', '에']]


In [10]:
text = "thisislongtext"
print(len(text), len(segment_naive(text)))

14 8192


In [13]:
text = "아버지가방에들어가신다" # Father goes into the bag or Father enters the room
print(len(text), len(segment_naive(text)))

11 1024


### Dynamic Programming

- Dynamic programming is a technique to solve a problem by breaking it into subproblems and storing the results of subproblems to avoid computing the same results again.
- The time complexity of dynamic programming is $O(n)$.
- For long strings, dynamic programming is much more efficient than the naive recursive algorithm.

```python
def segment(string, dictionary):
    if not string:
        return []
    for end in range(1, len(string) + 1):
        first, rest = string[:end], string[end:]
        if first in dictionary:
            return [first] + segment(rest, dictionary)
    return [string]
```

### Triangular Matrix

- The dynamic programming algorithm can be implemented using a triangular matrix.
- The tryangular matrix algorithm uses nested loops and a circular buffer to store the results of subproblems.
- A triangular matrix of parts with increasing length is generated and organized in a circular buffer.
- This allows a constant amount of memory to be used for the algorithm.


### Maximum Matching Algorithm

- If we have all known words in a dictionary, we can use the maximum matching algorithm to segment a sentence.
- The maximum matching algorithm is a greedy algorithm that finds the longest matching word from the dictionary.
- The algorithm is as follows:

    1. Find the longest matching word from the dictionary.
    2. If the word is found, add the word to the result and remove the word from the input.
    3. If the word is not found, add the first character to the result and remove the first character from the input.
    4. Repeat 1-3 until the input is empty.

### Unknown Words

- We can not rely on the dictionary to segment all words.
- There are uncommon words, new words, misspelled words, foreign words, proper nouns, slang words, etc.
- Even in these cases, we want to segment the words into meaningful parts.
- Therefore, we have to estimate the probability of any possible segmentation.

## Evaluation of Compositions

- Generally, we can evaluate a composition by calculating the probability of the composition.
- Word probabilities can be estimated from a corpus:

    $$
    P(w_i) = \frac{c(w_i)}{N}
    $$

    where $c(w_i)$ is the count of word $w_i$ and $N$ is the total number of words in the corpus.

- However, for unkonwn words, we have to use other criteria to evaluate the composition.
- At word boundary, the uncertainty of the segmentation increases.
- By measuring the uncertainty, we can evaluate the composition.

### Uncertainty of word boundaries

- The uncertainty of word boundaries can be measured by the entropy of the word boundary.
- {cite}`harris1970phoneme` said that if the uncertainty of successive tokens increases, the location is a word boundary.
- {cite}`feng2004accessor` proposed a statistical criterion called accessor variety (AV) to measure how likely a sub-sequence is a word, and then to find the best segmentation pattern that maximizes a target function of accessor variety and the length of the sub-sequence as variants. 
- {cite}`jin2006unsupervised` proposed branching entropy as another criterion for unsupervised segmentation.
- Both criteria share a similar assumption as in the fundamental work by Harris, 1970, that the uncertainty of successive tokens increases at word boundaries.
- The latter is the countinous version of the former.


![](../figs/intro_nlp/words/branching_entropy_uncertainty.png)


### Accessor Variety

- The accessor variety (AV) defines that the uncertainty of a sub-sequence is the number of different words that can be formed by adding a sub-sequence to the sub-sequence.
- For the forward accessor variety, it is the number of different words that can be formed by adding a sub-sequence to the right side of the sub-sequence.
- For the following sub-sequence, the forward accessor variety of `hope` is 2, because `hope` can be followed by `less` or `fully`.

    ```
    "hopeful": 100
    "hopeless": 80
    ```
- The backward accessor variety is the number of different words that can be formed by adding a sub-sequence to the left side of the sub-sequence.
- For example, the backward accessor variety of `less` is 3, because `hopeless`, `useless`, and `pointless` can be formed by adding `less` to the left side of `less`.

    ```
    "hopeless": 80
    "unless": 160
    "pointless": 70
    ```
- Depending on the language, the forward accessor variety or the backward accessor variety may be more suitable for segmentation.
- Threshold values can be used to determine the word boundaries.
- The threshold values can be determined by the corpus.

### Branching Entropy

**Assumption 1**: The uncertainty of successive tokens increases at word boundaries.

- Given a set of elements $X$ and a set of n-gram sequences $X_n$, the conditional entropy of an element occuring after an n-gram sequence $X_n$ is defined as:

    $$
    H(X|X_n) = -\sum_{x_n \in X_n} P(x_n) \sum_{x \in X} P(x|x_n) \log P(x|x_n)
    $$

    where $P(x) = P(X=x)$, $P(x|x_n) = P(X=x|X_n=x_n)$, and $P(X=x)$ indicates the probability of an element $x$ occuring in $X$.
- $H(X|X_n)$ decreases as $n$ increases, meaning that $X$ will become more predictable as $X_n$ becomes longer.
- The latter half of the equation, the entropy of $X$ given $X_n$, indicates the average information of branching out from a specific n-gram sequence $X_n$:

    $$
    H(X|X_n=x_n) = -\sum_{x \in X} P(x|x_n) \log P(x|x_n)
    $$

- This local entropy is the branching entropy of $X$ given $X_n$, and denoted as $h(x_n)$:

    $$
    h(x_n) = -\sum_{x \in X} P(x|x_n) \log P(x|x_n)
    $$


**Assumption 2**: If the entropy of successive tokens is high or increasing, the location is a word boundary.

- Generally, as the length of the n-gram sequence increases, the entropy of the n-gram sequence decreases.

    $$
    h(x_n) \geq h(x_{n-1})
    $$
    
- If $x_n$ is the prefix of $x_{n+1}$, the branching entropy of $x_n$ will likely be smaller than that of $x_{n+1}$:

    $$
    h(x_n) < h(x_{n+1})
    $$
    
- There are three boundary conditions to decide whether $x_n$ is a word boundary:

    1. $B_{max}$: If $h(x_n) > \text{val}_{max}$
    2. $B_{increase}$: $h(x_n) > h(x_{n-1})$
    3. $B_{ordinary}$: $h(x_n) > \text{val}_{ordinary}$

![](../figs/intro_nlp/words/branching_entropy.png)

## Word Segmentation in Practice

In [387]:
from ekorpkit import eKonf

eKonf.setLogger("WARNING")

cfg = eKonf.compose("corpus")
cfg.name = "fomc"
cfg.path.cache.uri = (
    "https://github.com/entelecheia/ekorpkit-book/raw/main/assets/data/fomc.zip"
)
cfg.data_dir = cfg.path.cached_path
cfg.auto.merge = True
fomc_corpus = eKonf.instantiate(cfg)
print(fomc_corpus)
texts = fomc_corpus.data[fomc_corpus.data.content_type == "fomc_statement"].text.tolist()

INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:setting environment variable CACHED_PATH_CACHE_ROOT to /workspace/.cache/cached_path
INFO:ekorpkit.base:setting environment variable KMP_DUPLICATE_LIB_OK to TRUE


Corpus : fomc


In [408]:
texts[0]

"Chairman Alan Greenspan announced today that the Federal Open Market Committee decided to increase slightly the degree of pressure on reserve positions. The action is expected to be associated with a small increase in short-term money market interest rates.\n\nThe decision was taken to move toward a less accommodative stance in monetary policy in order to sustain and enhance the economic expansion.\n\nChairman Greenspan decided to announce this action immediately so as to avoid any misunderstanding of the Committee's purposes, given the fact that this is the first firming of reserve market conditions by the Committee since early 1989."

In [366]:
texts = [
    "like apples",
    "likes pineapples apples",
    "dislikes Apple pineapple",
    "People dislike pineapples.",
    "An apple makes people healthy.",
    "Pine trees make pineapple",
    "Pineapple unhealthy",
    "likeness of apples",
    "dislikeness of pineapples",
    "health of pineapple",
    "healthyness of apple",
    "display of pineapples",
]

In [367]:
import re
import collections
from pprint import pprint


def normalize_word(word):
    # replace all non-alphanumeric characters at the end of the word with a space
    word = re.sub(r"[^a-zA-Z0-9]+$", " ", word)
    # replace all non-alphanumeric characters at the beginning of the word with a space
    word = re.sub(r"^[^a-zA-Z0-9]+", " ", word)
    return word.strip()


def pre_tokenize(text, lowercase=True):
    if lowercase:
        text = text.lower()
    # remove urls
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"\s+", " ", text)
    return [
        normalize_word(word) for word in text.split() if len(normalize_word(word)) > 0
    ]


In [388]:
whitespace_token = "▁"
empty_token = ""
max_fwd_len = 2
max_bwd_len = 2
character_freqs = collections.defaultdict(int)
subwords_freqs = collections.defaultdict(int)
word_freqs = collections.defaultdict(int)
forward_subwords_freqs = collections.defaultdict(int)
backward_subwords_freqs = collections.defaultdict(int)

for text in texts:
    words = pre_tokenize(text)
    for left_word, word, right_word in zip(
        [empty_token] + words[:-1], words, words[1:] + [empty_token]
    ):
        word_freqs[word] += 1
        for i in range(len(word)):
            character_freqs[word[i]] += 1
            # Loop through the subwords of length at least 2
            for j in range(i + 2, len(word) + 1):
                subwords_freqs[word[i:j]] += 1

        for j in range(min(len(word), max_fwd_len)):
            # forward_subword = word + whitespace_token + right_word[: j + 1]
            forward_subword = word + right_word[: j + 1]
            forward_subwords_freqs[forward_subword] += 1
        for j in range(min(len(word), max_bwd_len)):
            # backward_subword = left_word[-(j + 1) :] + whitespace_token + word
            backward_subword = left_word[-(j + 1) :] + word
            backward_subwords_freqs[backward_subword] += 1

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
print("Top 10 subwords: {}".format(sorted_subwords[:10]))
# sort forward subwords by frequency
sorted_forward_subwords = sorted(
    forward_subwords_freqs.items(), key=lambda x: x[1], reverse=True
)
print("Top 10 forward subwords: {}".format(sorted_forward_subwords[:10]))
# sort backward subwords by frequency
sorted_backward_subwords = sorted(
    backward_subwords_freqs.items(), key=lambda x: x[1], reverse=True
)
print("Top 10 backward subwords: {}".format(sorted_backward_subwords[:10]))
# subwords_freqs
# forward_subwords_freqs
# backward_subwords_freqs
print(len(subwords_freqs), len(sorted_forward_subwords), len(sorted_backward_subwords))


Top 10 subwords: [('in', 19653), ('th', 17728), ('on', 17147), ('an', 15225), ('er', 13980), ('at', 13068), ('re', 12853), ('ti', 12777), ('he', 12611), ('the', 11984)]
Top 10 forward subwords: [('thef', 2483), ('thec', 2166), ('theco', 1916), ('thefe', 1775), ('oft', 1310), ('monetaryp', 1267), ('monetarypo', 1267), ('ofth', 1227), ('int', 1134), ('fort', 1035)]
Top 10 backward subwords: [('sand', 2283), ('sof', 1808), ('efederal', 1781), ('hefederal', 1781), ('nthe', 1653), ('ecommittee', 1626), ('hecommittee', 1626), ('rthe', 1436), ('ethe', 1306), ('ypolicy', 1267)]
23821 14651 13723


In [390]:
bwd_tokens = list(character_freqs.items()) + sorted_subwords + sorted_backward_subwords
bwd_tokens = {token: freq for token, freq in bwd_tokens}
bwd_tokens = collections.Counter(bwd_tokens)

fwd_tokens = list(character_freqs.items()) + sorted_subwords + sorted_forward_subwords
fwd_tokens = {token: freq for token, freq in fwd_tokens}
fwd_tokens = collections.Counter(fwd_tokens)


In [391]:
class Trie:
    def __init__(self, end_symbol="<END>", direction="forward"):
        self.root = {}
        self.end_symbol = end_symbol
        self.direction = direction

    def add(self, word, value):
        if self.direction == "backward":
            # reverse the word
            word = word[::-1]
        node = self.root
        for ch in word:
            if ch not in node:
                node[ch] = {}
            node = node[ch]
        node[self.end_symbol] = value

    def get_value(self, word):
        if self.direction == "backward":
            # reverse the word
            word = word[::-1]
        node = self.root
        for ch in word:
            if ch not in node:
                return 0
            node = node[ch]
        if self.end_symbol not in node:
            return 0
        return node[self.end_symbol]

    def set_value(self, word, value):
        if self.direction == "backward":
            # reverse the word
            word = word[::-1]
        node = self.root
        for ch in word:
            if ch not in node:
                raise ValueError("word not in trie")
            node = node[ch]
        if self.end_symbol not in node:
            raise ValueError("word not in trie")
        node[self.end_symbol] = value

    def get_children(self, word):
        if self.direction == "backward":
            # reverse the word
            word = word[::-1]
        node = self.root
        for ch in word:
            if ch not in node:
                return []
            node = node[ch]
        children = node.copy()
        return children

    def get_values_of_children(self, word):
        children = self.get_children(word)
        values = []
        for child in children:
            if child == self.end_symbol:
                continue
            else:
                if self.end_symbol in children[child]:
                    values.append(children[child][self.end_symbol])
        return values
    
    def num_children(self, word):
        return len(self.get_children(word))

    def get_leafs(self, word):
        node = self.get_children(word)
        _ = node.pop(self.end_symbol, None)
        return self._get_leafs(node)
    
    def _get_leafs(self, node):
        if self.end_symbol in node:
            return [node[self.end_symbol]]
        leafs = []
        for child in node:
            leafs += self._get_leafs(node[child])
        return leafs

In [392]:
def initialize_trie(tokens, direction="forward"):
    trie = Trie(direction=direction)

    maxlen = 0
    for tok, val in tokens.items():
        trie.add(tok, val)
        maxlen = max(maxlen, len(tok))

    return trie, maxlen


In [393]:
fwd_trie, fwd_maxlen = initialize_trie(fwd_tokens, direction="forward")
bwd_trie, bwd_maxlen = initialize_trie(bwd_tokens, direction="backward")

In [395]:
# pprint(fwd_trie.get_children("pineapple"))
pprint(fwd_trie.get_children("monetary"))

{'<END>': 1439,
 'a': {'<END>': 144,
       'c': {'<END>': 11},
       'n': {'<END>': 131},
       'u': {'<END>': 2}},
 'b': {'<END>': 1, 'a': {'<END>': 1}},
 'c': {'<END>': 9, 'o': {'<END>': 9}},
 'e': {'<END>': 10, 'a': {'<END>': 10}},
 'p': {'<END>': 1267, 'o': {'<END>': 1267}},
 's': {'<END>': 7, 't': {'<END>': 7}},
 't': {'<END>': 1, 'i': {'<END>': 1}}}


In [412]:
import math
from scipy.special import digamma

def entropy(trie, word):
    leafs = trie.get_leafs(word)
    val = trie.get_value(word)
    logsum = digamma(sum(leafs) + val)
    entropy = 0
    for freq in leafs:
        logprob = digamma(freq) - logsum
        entropy += math.exp(logprob) * logprob
    return -1 * entropy

word = "rates"
# word = "pineapples"
# word = "people"
# word = "unhealthy"
# word = "healthyness"
# word = "dislikeness"
# word = "likes"s
for i in range(1, len(word)+1):
    print(word[:i], entropy(fwd_trie, word[:i]))

for i in range(1, len(word)+1):
    print(word[-i:], entropy(bwd_trie, word[-i:]))

r 1.5279808395403343
ra 1.2169536856123169
rat 0.4515554246117919
rate 1.255306316910514
rates 1.4232282798426306
s 1.2349366144473954
es 1.4550346892773653
tes 0.7783313449194234
ates 0.9638677718791596
rates 0.8214645612592879


## References

- [Uncertanty to word boundary; Accessor Variety & Branching Entropy](https://lovit.github.io/nlp/2018/04/09/branching_entropy_accessor_variety/)
- [Fast Word Segmentation of Noisy Text](https://medium.com/towards-data-science/fast-word-segmentation-for-noisy-text-2c2c41f9e8da)